In [1]:
#TODO this should be in requirments.txt

%%sh
pip install pycaret
pip install python-dotenv
pip install ydata-profiling
pip install shap
pip -q install --upgrade stepfunctions
pip install s3fs

In [4]:
#TODO remove unused imports
#TODO wrap this whole script in a retraining.py file to be scheduled on a regular basis in e.g Glue or Lambda (depending how long it takes to run it)

import boto3
import sagemaker
import uuid
import logging
import stepfunctions
import os

from sagemaker.amazon.amazon_estimator import get_image_uri
from stepfunctions import steps
from stepfunctions.inputs import ExecutionInput
from stepfunctions.workflow import Workflow

from sagemaker import get_execution_role
from dotenv import load_dotenv
from load_data import load_data
from transform_data import split_data


In [5]:
# Variables Setup Stage
load_dotenv(".env")
role = get_execution_role()

# Env variables
data_location_s3 = os.getenv("data_location_s3")
algorithm_choice = os.getenv("algorithm_choice")
target = os.getenv("target")
endpoint_name = os.getenv("endpoint_name")
model_name = os.getenv("model_name")
data_location = "s3://{}".format(data_location_s3)
instance_type = os.getenv("instance_type")
model_instance_count = int(os.getenv("model_instance_count"))
image_uri = os.getenv("ecr_repo_uri")
tuning_metric = os.getenv("tuning_metric")

#TODO change to log instead of print
print(
    data_location_s3,
    algorithm_choice,
    target,
    endpoint_name,
    model_name,
    data_location,
    instance_type,
    image_uri,
    tuning_metric,
    role
)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/lanre.bakare/Library/Application Support/sagemaker/config.yaml
streaming-data-platform-ml-data/ethan_data.csv classification y classification-proba-endpoint banking-classification s3://streaming-data-platform-ml-data/ethan_data.csv ml.m4.xlarge mlops-classification-repo AUC arn:aws:iam::135544376709:role/aws-reserved/sso.amazonaws.com/eu-west-1/AWSReservedSSO_AdministratorAccess_a474e246441df619


In [4]:
stepfunctions.set_stream_logger(level=logging.INFO)
bucket = "streaming-data-platform-ml-data" #TODO remove any hardcoded variables like this and pass them from Terraform
prefix = "step_function"
bucket_path = 's3://{}/{}/'.format(bucket, prefix)
region = boto3.Session().region_name
session = sagemaker.Session()

In [5]:
FILE_DATA = load_data(data_location)
print(FILE_DATA.head())

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)
/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:279: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


   age  job  education  default  balance  housing  loan  y
0   32    7          2        1     -238        1     0  0
1   34    4          2        0     -478        1     1  0
2   32    3          2        0      266        1     0  0
3   36    7          2        1       13        0     1  0
4   23   11          2        0      486        0     0  0
   age  job  education  default  balance  housing  loan  y
0   32    7          2        1     -238        1     0  0
1   34    4          2        0     -478        1     1  0
2   32    3          2        0      266        1     0  0
3   36    7          2        1       13        0     1  0
4   23   11          2        0      486        0     0  0


In [6]:
print(len(FILE_DATA)) #TODO add a more meaningful log message

55818
55818


In [7]:
#TODO remove no need for this
FILE_DATA = FILE_DATA.sample(frac=1)
FILE_DATA.head()

,age,job,education,default,balance,housing,loan,y
25966,31,2,3,0,83,0,0,0
25244,29,3,4,0,1729,1,0,0
5560,32,2,3,0,3237,1,0,0
48206,44,3,3,0,5468,0,0,1
53177,33,7,2,0,476,1,0,1


,age,job,education,default,balance,housing,loan,y
25966,31,2,3,0,83,0,0,0
25244,29,3,4,0,1729,1,0,0
5560,32,2,3,0,3237,1,0,0
48206,44,3,3,0,5468,0,0,1
53177,33,7,2,0,476,1,0,1


In [8]:
from sklearn.model_selection import train_test_split #TODO include this in other imports on top

train_and_val_data, test_data = split_data(FILE_DATA, shuffle=True)

In [9]:
train_data, validation_data = split_data(train_and_val_data, shuffle=True)

In [10]:
print(len(train_data), len(validation_data), len(test_data)) #TODO add a more meaningful log message

35723 8931 11164
35723 8931 11164


In [11]:
#TODO move this to the top where we declare other variables

FILE_TRAIN = "train.csv"
FILE_TEST = "test.csv"
FILE_VALIDATION = "validation.csv"

In [12]:
#TODO move this to the top where we declare other variables
train_s3_file = os.path.join(prefix, FILE_TRAIN)
test_s3_file = os.path.join(prefix, FILE_TEST)
validation_s3_file = os.path.join(prefix, FILE_VALIDATION)
print(train_s3_file, test_s3_file, validation_s3_file)

step_function/train.csv step_function/test.csv step_function/validation.csv
step_function/train.csv step_function/test.csv step_function/validation.csv


In [13]:
from io import StringIO #TODO include this in other imports on top

# Upload the three files to Amazon S3
#TODO wrap this in a function to avoid duplication
csv_buffer = StringIO()
train_data.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, train_s3_file).put(Body=csv_buffer.getvalue())

csv_buffer = StringIO()
test_data.to_csv(csv_buffer, index=False, header=False)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, test_s3_file).put(Body=csv_buffer.getvalue())

csv_buffer = StringIO()
validation_data.to_csv(csv_buffer, index=False)
s3_resource = boto3.resource("s3")
s3_resource.Object(bucket, validation_s3_file).put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'F43T67Y6ZVAZZ8Q5',
  'HostId': 'BytTqT8C3l4gAQpCXpN/66K60U1/9a/YNJkJeMM9yGNysUuHEhjA+iCK/fSpkj4j18nXinLGQ2c=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BytTqT8C3l4gAQpCXpN/66K60U1/9a/YNJkJeMM9yGNysUuHEhjA+iCK/fSpkj4j18nXinLGQ2c=',
   'x-amz-request-id': 'F43T67Y6ZVAZZ8Q5',
   'date': 'Fri, 09 Aug 2024 15:19:22 GMT',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:eu-west-1:135544376709:key/30fbba55-c738-4f17-a525-65cb9e22bd2c',
   'etag': '"47525e86fe4f3dd5882064953ad80ceb"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"47525e86fe4f3dd5882064953ad80ceb"',
 'ServerSideEncryption': 'aws:kms',
 'SSEKMSKeyId': 'arn:aws:kms:eu-west-1:135544376709:key/30fbba55-c738-4f17-a525-65cb9e22bd2c'}

{'ResponseMetadata': {'RequestId': 'F43T67Y6ZVAZZ8Q5',
  'HostId': 'BytTqT8C3l4gAQpCXpN/66K60U1/9a/YNJkJeMM9yGNysUuHEhjA+iCK/fSpkj4j18nXinLGQ2c=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'BytTqT8C3l4gAQpCXpN/66K60U1/9a/YNJkJeMM9yGNysUuHEhjA+iCK/fSpkj4j18nXinLGQ2c=',
   'x-amz-request-id': 'F43T67Y6ZVAZZ8Q5',
   'date': 'Fri, 09 Aug 2024 15:19:22 GMT',
   'x-amz-server-side-encryption': 'aws:kms',
   'x-amz-server-side-encryption-aws-kms-key-id': 'arn:aws:kms:eu-west-1:135544376709:key/30fbba55-c738-4f17-a525-65cb9e22bd2c',
   'etag': '"47525e86fe4f3dd5882064953ad80ceb"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"47525e86fe4f3dd5882064953ad80ceb"',
 'ServerSideEncryption': 'aws:kms',
 'SSEKMSKeyId': 'arn:aws:kms:eu-west-1:135544376709:key/30fbba55-c738-4f17-a525-65cb9e22bd2c'}

In [14]:
train_s3_file = 's3://{}/{}'.format(bucket, train_s3_file)
validation_s3_file = 's3://{}/{}'.format(bucket, validation_s3_file)
test_s3_file = 's3://{}/{}'.format(bucket, test_s3_file)
output_s3  = 's3://{}/{}/{}/'.format(bucket, prefix, 'output')
print(train_s3_file, validation_s3_file, test_s3_file, output_s3)

s3://streaming-data-platform-ml-data/step_function/train.csv s3://streaming-data-platform-ml-data/step_function/validation.csv s3://streaming-data-platform-ml-data/step_function/test.csv s3://streaming-data-platform-ml-data/step_function/output/
s3://streaming-data-platform-ml-data/step_function/train.csv s3://streaming-data-platform-ml-data/step_function/validation.csv s3://streaming-data-platform-ml-data/step_function/test.csv s3://streaming-data-platform-ml-data/step_function/output/


In [16]:
sagemaker_execution_role = role
xgb = sagemaker.estimator.Estimator(
    get_image_uri(region, 'xgboost', repo_version='0.90-2'), #TODO pass the image name where we define other variables at the top
    sagemaker_execution_role, 
    train_instance_count = 1, 
    train_instance_type = 'ml.m4.4xlarge', #TODO this should be the same training instance that we pass from Terraform
    output_path = output_s3,
    sagemaker_session = session
)

xgb.set_hyperparameters(
    objective = 'reg:linear', #TODO the objective should be the same as algorithm type from Terraform. Also we should do some basic hyperparameter tuning
    num_round = 50,
    max_depth = 5,
    eta = 0.2,
    gamma = 4,
    min_child_weight = 6,
    subsample = 0.7,
    silent = 0
)

In [17]:
execution_input = ExecutionInput(schema={
    'JobName': str, 
    'ModelName': str,
    'EndpointName': str
})

In [18]:
train_s3_file

's3://streaming-data-platform-ml-data/step_function/train.csv'

's3://streaming-data-platform-ml-data/step_function/train.csv'

In [19]:
training_step = steps.TrainingStep(
    'Train Step', 
    estimator=xgb,
    data={
        'train': sagemaker.inputs.TrainingInput(train_s3_file, content_type='csv'),
        'validation': sagemaker.inputs.TrainingInput(validation_s3_file, content_type='csv')
    },
    job_name=execution_input['JobName']
)

In [20]:
model_step = steps.ModelStep(
    'Save model',
    model=training_step.get_expected_model(),
    model_name=execution_input['ModelName']  
)

In [ ]:
#TODO ideally this should perform the same transformations as here https://github.com/crederauk/terraform-aws-mlops-module/blob/e3aa13fdbc02ce328b45fa466c07e3acc9e8632a/mlops_ml_models/transform_data.py#L39
# transform_step = steps.TransformStep(
#     'Transform Input Dataset',
#     transformer=xgb.transformer(
#         instance_count=1,
#         instance_type='ml.m5.large'
#     ),
#     job_name=execution_input['JobName'],     
#     model_name=execution_input['ModelName'], 
#     data=test_s3_file,
#     content_type='text/csv'
# )

In [22]:
endpoint_config_step = steps.EndpointConfigStep(
    "Create Endpoint Config",
    endpoint_config_name=execution_input['ModelName'],
    model_name=execution_input['ModelName'],
    initial_instance_count=1,
    instance_type='ml.m5.large' #TODO this should be the same endpoint instance type that we pass from Terraform
)

In [23]:
endpoint_step = steps.EndpointStep(
    "Create Endpoint",
    endpoint_name=execution_input['EndpointName'],
    endpoint_config_name=execution_input['ModelName']
)

In [25]:
workflow_definition = steps.Chain([
    training_step,
    model_step,
#     transform_step, #TODO bring back once we have the transform step working
    endpoint_config_step,
    endpoint_step
])

In [26]:
from time import strftime, gmtime #TODO import on the top as the other libraries
timestamp = strftime('%d-%H-%M-%S', gmtime()) #TODO define together with other variables at the top

workflow = Workflow(
    name='{}-{}'.format('MyTrainTransformDeploy_v1', timestamp), #TODO the name should be model_name variable from terraform with some version number
    definition=workflow_definition,
    role=role,
    execution_input=execution_input
)

In [27]:
workflow.render_graph()

In [28]:
workflow.create()

[INFO] Workflow created successfully on AWS Step Functions.
[INFO] Workflow created successfully on AWS Step Functions.


'arn:aws:states:eu-west-1:135544376709:stateMachine:MyTrainTransformDeploy_v1-09-15-19-30'

'arn:aws:states:eu-west-1:135544376709:stateMachine:MyTrainTransformDeploy_v1-09-15-19-30'

In [29]:
execution = workflow.execute(
    inputs={
        #TODO those names should come from Terraform
        'JobName': 'regression-{}'.format(uuid.uuid1().hex), # Each Sagemaker Job requires a unique name
        'ModelName': 'regression-{}'.format(uuid.uuid1().hex), # Each Model requires a unique name,
        'EndpointName': 'regression-{}'.format(uuid.uuid1().hex) # Each Endpoint requires a unique name,
    }
)

[INFO] Workflow execution started successfully on AWS Step Functions.
[INFO] Workflow execution started successfully on AWS Step Functions.


In [31]:
execution.render_progress()